In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 14:57:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def display(spark_df, rows=5):
    '''takes in a spark dataframe and displays a given number of rows using pandas formatting (for convenience)'''
    return spark_df.limit(rows).toPandas()

#### 1. Read the case, department, and source data into their own spark dataframes.

In [3]:
case = spark.read.csv('case.csv', 
                       sep=',', 
                       header=True, 
                       inferSchema=True)

dept = spark.read.csv('dept.csv', 
                       sep=',', 
                       header=True, 
                       inferSchema=True)

source = spark.read.csv('source.csv', 
                         sep=',', 
                         header=True,
                         inferSchema=True)

#### 2. Let's see how writing to the local disk works in spark:
#### 2a. Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json



In [5]:
source.write.json('source_json')
source.write.csv('source_csv')

#### 2b. Inspect your folder structure. What do you notice?

source_csv and source_json are directories, not files

In [6]:
ls

case.csv                    source_json/
dept.csv                    spark-warehouse/
history.txt                 spark101.ipynb
source.csv                  spark_mini_exercises.ipynb
source_csv/                 wrangle_notebook.ipynb


#### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.


In [7]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [8]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [9]:
fmt = 'M/d/yy H:mm'

case = (
    case.withColumn('case_id', case.case_id.cast('string'))
        .withColumn('case_opened_date', F.to_timestamp('case_opened_date', fmt))
        .withColumn('case_closed_date', F.to_timestamp('case_closed_date', fmt))
        .withColumnRenamed('SLA_due_date', 'case_due_date')
        .withColumn('case_due_date', F.to_timestamp('case_due_date', fmt))
        .withColumn('case_closed', F.expr('case_closed == "YES"'))
        .withColumn('case_late', F.expr('case_late == "YES"'))
        .withColumn('council_district', case.council_district.cast('string'))
        .withColumn('case_age', F.datediff(F.current_timestamp(), 'case_opened_date'))
        .withColumn('days_to_closed', F.datediff('case_closed_date', 'case_opened_date'))
        .withColumn('case_lifetime', F.when(F.expr('! case_closed'), F.col('case_age'))
                                      .otherwise(F.col('days_to_closed')))
)  
case.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

In [10]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1600                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
only showing top 1 row



#### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [11]:
open_cases = case.filter(~F.col('case_closed'))

In [12]:
case.select(F.max('case_age')).show()

+-------------+
|max(case_age)|
+-------------+
|         1965|
+-------------+



#### 2. How many Stray Animal cases are there?

In [13]:
case.filter(F.col('service_request_type') == 'Stray Animal').count()

26760

#### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?


In [14]:
field_ops_cases = case.filter(F.col('dept_division') == 'Field Operations')
field_ops_cases.filter(F.col('service_request_type') != 'Officer Standby').count()

113902

#### 4. Convert the council_district column to a string column.

see column transformations in the first #3 above

#### 5. Extract the year from the case_closed_date column.

In [15]:
case = case.withColumn('year_closed', F.year('case_closed_date'))

#### 6. Convert num_days_late from days to hours in new columns num_hours_late.


In [16]:
case = case.withColumn('num_hours_late', F.col('num_days_late')*24)

#### 7. Join the case data with the source and department data.

In [17]:
df = (
    case.join(source, on='source_id', how='left')
        .join(dept, on='dept_division', how='left')
        .drop('dept_name')
        .withColumnRenamed('standardized_dept_name', 'department')
        .withColumn('dept_subject_to_SLA', (F.col('dept_subject_to_SLA') == 'YES'))
)

#### 8. Are there any cases that do not have a request source?


In [18]:
df.filter(
          F.col('source_id').isNull()
        | F.isnan('source_id')
        | (F.col('source_id') == '')
).count()

0

#### 9. What are the top 10 service request types in terms of number of requests?


In [19]:
(
    df.groupby('service_request_type')
      .count()
      .sort(F.col('count').desc())
      .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



#### 10. What are the top 10 service request types in terms of average days late?

In [20]:
(
    df.groupby('service_request_type')
      .agg(F.mean('num_days_late'))
      .sort(F.col('avg(num_days_late)').desc())
      .show(10, truncate=False)
)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.95636210420943|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.11255641500003|
|Zoning: Recycle Yard                  |135.92851612479797|
|Donation Container Enforcement        |131.75610506358706|
|License Requied Used Mattress Sales   |128.79828704142858|
|Traffic Signal Graffiti               |101.79846062200002|
|Complaint                             |72.8705023031169  |
+--------------------------------------+------------------+
only showing top 10 rows



#### 11. Does number of days late depend on department?

In [21]:
(
    df.groupby('department')
      .agg(F.mean('num_days_late'))
      .sort(F.col('avg(num_days_late)').desc())
      .show()
)

+--------------------+-------------------+
|          department| avg(num_days_late)|
+--------------------+-------------------+
|    Customer Service|  59.73709149630077|
|         Solid Waste| -2.200057513672164|
|        Metro Health| -4.911766979607004|
|  Parks & Recreation| -5.251521960055145|
|Trans & Cap Impro...|-20.612837354052708|
|DSD/Code Enforcement| -38.36938892614506|
|Animal Care Services|-226.51783940550334|
|        City Council|               null|
+--------------------+-------------------+



#### 12. How do number of days late depend on department and request type?

In [22]:
(
    df.groupby('department', 'service_request_type')
      .agg(F.mean('num_days_late'))
      .sort('department', 'service_request_type')
      .show(truncate=False)
)

+--------------------+--------------------------------------+--------------------+
|department          |service_request_type                  |avg(num_days_late)  |
+--------------------+--------------------------------------+--------------------+
|Animal Care Services|Aggressive Animal(Critical)           |16.69636881189224   |
|Animal Care Services|Aggressive Animal(Non-Critical)       |2.645033883163268   |
|Animal Care Services|Animal Bite(Critical)                 |0.019096834813110173|
|Animal Care Services|Animal Bite(Non-Critical)             |-2.505667547973447  |
|Animal Care Services|Animal Cruelty(Critical)              |-0.06219277353522766|
|Animal Care Services|Animal Neglect                        |7.332927878069383   |
|Animal Care Services|Animal Permits Request                |22.19792271447681   |
|Animal Care Services|City Council Animal Request           |-1.5475013634821913 |
|Animal Care Services|Injured Animal(Critical)              |-0.06529248474639532|
|Ani

In [23]:
display(df, rows=3)

,dept_division,source_id,case_id,case_opened_date,case_closed_date,case_due_date,case_late,num_days_late,case_closed,service_request_type,...,request_address,council_district,case_age,days_to_closed,case_lifetime,year_closed,num_hours_late,source_username,department,dept_subject_to_SLA
0,Field Operations,svcCRMLS,1014127332,2018-01-01 00:42:00,2018-01-01 12:29:00,2020-09-26 00:42:00,False,-998.508762,True,Stray Animal,...,"2315 EL PASO ST, San Antonio, 78207",5,1600,0,0,2018,-23964.210278,svcCRMLS,Animal Care Services,True
1,Storm Water,svcCRMSS,1014127333,2018-01-01 00:46:00,2018-01-03 08:11:00,2018-01-05 08:30:00,False,-2.012604,True,Removal Of Obstruction,...,"2215 GOLIAD RD, San Antonio, 78223",3,1600,2,2,2018,-48.302500,svcCRMSS,Trans & Cap Improvements,True
2,Storm Water,svcCRMSS,1014127334,2018-01-01 00:48:00,2018-01-02 07:57:00,2018-01-05 08:30:00,False,-3.022338,True,Removal Of Obstruction,...,"102 PALFREY ST W, San Antonio, 78223",3,1600,1,1,2018,-72.536111,svcCRMSS,Trans & Cap Improvements,True


22/05/20 19:24:34 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1630955 ms exceeds timeout 120000 ms
22/05/20 19:24:34 WARN SparkContext: Killing executors is not supported by current scheduler.
